In [3]:
%%time

import pandas as pd
import numpy
import os
import tarfile
import dask
from dask.distributed import Client, progress
client = Client(threads_per_worker=1, n_workers=2)

def genotype_compare(geno_a, geno_b):
  (a1, a2) = geno_a.split("|")
  (b1, b2) = geno_b.split("|")

  if a1 == b1 and a2 == b2:
    return 1
  elif a1 == b2 and a2 == b1:
    return 1
  elif a1 == b1 or a2 == b2:
    return .5
  else:
    return 0

def test_genotype_compare():
    assert genotype_compare("1|1", "1|1") == 1
    assert genotype_compare("0|0", "0|0") == 1
    assert genotype_compare("1|0", "1|0") == 1
    assert genotype_compare("0|1", "0|1") == 1
    assert genotype_compare("1|0", "0|1") == 1

    assert genotype_compare("1|1", "0|1") == 0.5
    assert genotype_compare("0|0", "0|1") == 0.5
    assert genotype_compare("1|1", "1|0") == 0.5
    assert genotype_compare("0|0", "1|0") == 0.5

    assert genotype_compare("1|1", "0|0") == 0.0
    assert genotype_compare("0|0", "1|1") == 0.0

test_genotype_compare()

def make_matrix(row, samples):
  array = numpy.zeros(shape=[len(samples), len(samples)])
  for j, sample_a in enumerate(samples):
    for k, sample_b in enumerate(samples):
      array[j,k] = genotype_compare(row[sample_a], row[sample_b])
  return array

def export_matrix(vcf_file, skip_rows):
    # todo: handle skiprows?

    vcf_df = pd.read_csv(vcf_file, sep="\t", skiprows=skip_rows, dtype='string')
    samples = list(vcf_df.columns)[9:]
    vcf_df['matrix'] = vcf_df.apply(lambda row: make_matrix(row, samples), axis=1)

    vcf_file_base = '.'.join(os.path.basename(vcf_file).split('.')[:-2])

    tar_file_name = f"{vcf_file_base}.row_matrices.tar.gz"

    # create tar file
    with tarfile.open(tar_file_name, "w:gz") as tar:
        for row in range(0,vcf_df.shape[0]):
          row_df = pd.DataFrame(vcf_df.iloc[row]['matrix'], columns=samples, index=samples)
          id = vcf_df.iloc[row]['ID']
          filename = f"{vcf_file_base}.{id}.matrix.tsv"
          row_df.to_csv(filename, sep="\t")
          tar.add(filename)
          os.remove(filename)

    tar.close()

    return tar_file_name

# This is just a dummy function for dask reasons
def join(a):
    print(a)

def generate_matrices():
    #phenotypes = ["max_height_cm", "max_growth_cm_gdd"]
    # pvalues = ["p0001", "p0005", "p001"]
    phenotypes = ["max_height_cm", "max_growth_cm_gdd"]
    pvalues = ["p0001", "p0005"]


    delayed_results = []

    for phenotype in phenotypes:
        for pvalue in pvalues:
            export_matrix(f"../vcf/sorghum.filtered.season4.season6.{phenotype}_{pvalue}.vcf.gz", skip_rows=84)
            export_matrix(f"../vcf/sorghum.filtered.season4.season6.{phenotype}_{pvalue}_qtl.vcf.gz", skip_rows=85)
#             delayed_results.append(dask.delayed(export_matrix)(f"../vcf/sorghum.filtered.season4.season6.{phenotype}_{pvalue}.vcf.gz", skip_rows=84))
#             delayed_results.append(dask.delayed(export_matrix)(f"../vcf/sorghum.filtered.season4.season6.{phenotype}_{pvalue}_qtl.vcf.gz", skip_rows=85))

#     x = dask.delayed(join)(delayed_results)
#     x.compute()

generate_matrices()

/opt/homebrew/lib/python3.10/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 63975 instead
  warnings.warn(


CPU times: user 1h 8min 54s, sys: 1min 25s, total: 1h 10min 20s
Wall time: 1h 30min 7s


2022-10-28 21:48:49,311 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 1 memory: 44 MB fds: 29>>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/opt/homebrew/lib/python3.10/site-packages/distributed/system_monitor.py", line 133, in update
    net_ioc = psutil.net_io_counters()
  File "/opt/homebrew/lib/python3.10/site-packages/psutil/__init__.py", line 2114, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory
2022-10-29 03:18:52,438 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 2 memory: 46 MB fds: 29>>
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.10/site-packages/tornado/ioloop.py", line 905, in _run
    return self.callback()
  File "/opt/homebrew/lib/python3.10/site